<a href="https://colab.research.google.com/github/Kussil/Financial_Sentiment_LLM/blob/main/03_Sentiment_Analysis/Sentiment_Analysis_(FinBERT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Add Sentiment Analysis

In [ ]:
#installs
!pip install -q tqdm
!pip install -q accelerate
!pip install -q bitsandbytes
!pip install -q transformers
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which i

In [ ]:
#imports
import os
from google.colab import userdata
import pandas as pd
import math
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import torch
from tqdm import tqdm
from datasets import Dataset, DatasetDict

### Import Files From GitHub

In [ ]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/CVX_Rice_project.git

Cloning into 'CVX_Rice_project'...
remote: Enumerating objects: 1674, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 1674 (delta 47), reused 65 (delta 38), pack-reused 1595
Receiving objects: 100% (1674/1674), 352.91 MiB | 8.28 MiB/s, done.
Resolving deltas: 100% (1130/1130), done.
Updating files: 100% (1105/1105), done.


In [ ]:
invest_df1 = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/CVX_Rice_project/02_Cleaned_Data/SEC_Filings.csv')

df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
df['Article Index'] = df.index
df['Date'] = pd.to_datetime(df['Date'], format='mixed')
df['Date']= pd.to_datetime(df['Date']).dt.date
df

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,1
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,2
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,3
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,4
...,...,...,...,...,...,...,...,...
10121,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN,10121
10122,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10122
10123,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN,10123
10124,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN,10124


### Chunk Text for FinBERT, < 512 token input

In [ ]:
#Chunk text
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')

#Set token limit based on model constraint - BERT is 512
token_limit = 500

#Create function to chunk data
def chunk_contents(x):
  '''
  This function chunks the inputs from a pandas dataframe based on a provided
  token limit. For FinBert this limit is set to 500 because BERT max is 512
  and decoding creates additional tokens

  Parameters:
  x (str): Article Text from a pandas dataframe of all the financial documents

  Returns
  chunk_list (list of str): A python list of the Article Text broken into
  chunks, thes chunks are in a list inside a single row of the source dataframe
  '''
  x_str = str(x)
  tokens = tokenizer(x_str)['input_ids']
  token_count = len(tokens)
  chunk_count = math.ceil(token_count / token_limit)
  chunk_list = []
  tokenized_chunk_list = []
  for i in range(chunk_count):
    tokenized_chunk = tokens[i*token_limit:(i+1)*token_limit]
    chunk_list.append(tokenizer.decode(tokenized_chunk))
  return chunk_list

#Write chunks to dataframe
tqdm.pandas()
df['Text Chunks'] = df['Article Text'].progress_apply(chunk_contents)
df

100%|██████████| 10126/10126 [24:12<00:00,  6.97it/s]


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index,Text Chunks
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"[[CLS] stock report | may 16, 2024 | nysesymbo..."
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN,1,"[[CLS] stock report | may 14, 2024 | nysesymbo..."
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN,2,"[[CLS] stock report | may 11, 2024 | nysesymbo..."
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN,3,"[[CLS] stock report | may 11, 2024 | nysesymbo..."
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN,4,"[[CLS] stock report | may 07, 2024 | nysesymbo..."
...,...,...,...,...,...,...,...,...,...
10121,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN,10121,[[CLS] question and answer operator [ operator...
10122,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10122,[[CLS] question and answer operator [ operator...
10123,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN,10123,[[CLS] question and answer operator [ operator...
10124,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN,10124,[[CLS] question and answer operator [ operator...


### Apply FinBERT Sentiment Analysis
Create Neurtal, Positive, Negative matrix for each chunk

In [ ]:
#split into chunks and create new dataframe
df_chunk = df.explode('Text Chunks').reset_index(drop=True)

#Set NaN Tickers to blanks
df_chunk['Ticker'] = df_chunk['Ticker'].fillna('')
df_chunk

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Article Index,Text Chunks
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"[CLS] stock report | may 16, 2024 | nysesymbol..."
1,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"797 e 1, 804 e 7, 155 2024 1, 536 e 1, 705 e 1..."
2,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,". 12 - 1. 16 uin q1 2024, mro ’ s capex ( $ 60..."
3,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,be relied as such. mro with revenue growth of ...
4,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN,0,"texas, e & p. 77024 - 2217 the company ’ s est..."
...,...,...,...,...,...,...,...,...,...
122683,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10125,to the buyer in full. we're not - - there's no...
122684,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10125,we're not - - these investment decisions are n...
122685,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10125,received a distribution this quarter from apln...
122686,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN,10125,beginning in the third or fourth quarter this ...


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

sentiment_checkpoint = 'ProsusAI/finbert'
softmax = torch.nn.Softmax(dim=1)

######## Sentiment Analysis ############################
finbert = BertForSequenceClassification.from_pretrained(sentiment_checkpoint)#,num_labels=3)
finbert.to(device)
tokenizer = BertTokenizer.from_pretrained(sentiment_checkpoint)

labels = {0:'neutral', 1:'positive',2:'negative'}

def finbert_sentiment(chunk):
  '''
  This function runs the FinBERT sentiment analysis model. This model runs 1
  input at a time because the tokenizer on CPU is the slowest part so batching
  through FinBERT on GPU is negligble time improvement

  Parameters:
  chunk (str): a text chunk from the chunked dataframe

  Returns:
  content_sentiment_list (list of floats): returns a list of the sentiment
  scores
  '''
  with torch.no_grad():
    content_sentiment_list = []
    inputs = tokenizer(chunk, return_tensors="pt", padding=True, truncation=True)
    inputs.to(device)
    outputs = finbert(**inputs)[0]
    softmax_outputs = softmax(outputs)[0]
    content_sentiment_list.append([softmax_outputs[0].item(), softmax_outputs[1].item(), softmax_outputs[2].item()])
  return content_sentiment_list

tqdm.pandas()
df_chunk[['Neutral', 'Positive', 'Negative']] = df_chunk['Text Chunks'].progress_apply(finbert_sentiment)
df


100%|██████████| 122688/122688 [1:15:24<00:00, 27.12it/s]


ValueError: Columns must be same length as key

Create Classifications for each chunk

In [ ]:
######## Classification Analysis ############################
#Check if a GPU is available
device = 0 if torch.cuda.is_available() else -1

#Initialize the zero-shot classification pipeline with the specified device
bartclassifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

#Define the candidate labels
candidate_labels = ['acquisition', 'earnings', 'production', 'exploration', 'green energy']

def bart_classifier(chunk_list, batch_size):
    with torch.no_grad():
        #Perform classification in batches
        outputs = bartclassifier(chunk_list['Text Chunks'], candidate_labels, multi_label=True, batch_size=batch_size)

        #Initialize a list to store the scores
        batch_scores = []
        for output in outputs:
            batch_scores.append(output['scores'])

    return batch_scores

#Convert the 'Text Chunks' column to a list and create a Dataset
chunk_list = df_chunk['Text Chunks'].tolist()
chunk_list = pd.DataFrame({'Text Chunks': chunk_list})
chunk_list = Dataset.from_pandas(chunk_list)

#Create a DatasetDict
ds = DatasetDict({'predict': chunk_list})

# Determine the number of chunks and the batch size
count_chunks = len(chunk_list)
batch_size = 10
chunk_class_list =[]

# Process chunks in batches
for i in tqdm(range(int(count_chunks / batch_size) + 1), desc="Loading..."):
    class_list = bart_classifier(ds['predict'].select(range(i * batch_size, min((i + 1) * batch_size, count_chunks))), batch_size)
    chunk_class_list.extend(class_list)

# Add the classification scores to the DataFrame
df_chunk[candidate_labels] = pd.DataFrame(chunk_class_list, index=df_chunk.index)

df_chunk

Merge chunk sentiments into text sentiment

In [ ]:
#Aggregate chunk sentements and classifications back into Article level dataframe
grouped_df = df_chunk.groupby(['Article Index','Ticker','Source','Date','Article Headline','Article Text','URL'])[['Neutral', 'Positive', 'Negative','acquisition', 'earnings', 'production', 'exploration', 'green energy']].mean().reset_index()

grouped_df

In [ ]:
#Drop article text from chunk dataframe
df_chunk = df_chunk.drop('Article Text', axis=1)

df_chunk

Write to GitHub Repo

In [ ]:
#Save csv locally
grouped_df.to_csv('/content/sample_data/sentiment_data.csv', index=True)
df_chunk.to_csv('/content/sample_data/sentiment_chunkdata.csv', index=True)

In [ ]:
#Copy CSV to GitHub Repository
!cp /content/sample_data/sentiment_data.csv /content/CVX_Rice_project/03_Sentiment_Analysis/
!cp /content/sample_data/sentiment_chunkdata.csv /content/CVX_Rice_project/03_Sentiment_Analysis/

In [ ]:
#Commit and Push Changes
!git config --global user.email "bw58@rice.edu"
%cd /content/CVX_Rice_project/03_Sentiment_Analysis/
!git add sentiment_data.csv
!git commit -m "Add sentiment_data.csv"
!git add sentiment_chunkdata.csv
!git commit -m "Add sentiment_chunkdata.csv"
!git push origin main

In [ ]:
#Remove github clone
!rm -rf /content/CVX_Rice_project/